In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

/home/vicrrs/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import getpass
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
torch.random.manual_seed(42)

### Definição de Token

In [4]:
os.environ["HF_TOKEN"] = getpass.getpass()

In [5]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [6]:
#model = AutoModelForCausalLM.from_pretrained(
#    id_model,
#    trust_remote_code=True,
#    attn_implementation="eager",
#    torch_dtype=torch.float16,
#).to("cuda")

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=False,
    attn_implementation="eager",
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.03s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [8]:
next(model.parameters()).device


device(type='cuda', index=0)

### Tokenizer
Converte texto bruto em tokens

In [9]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

### Criacao pipeline

In [10]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

Device set to use cuda:0


### Parametro geracao de texto

In [11]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 ate 0.9
    "do_sample": True
}

In [12]:
prompt = "Explique o que é machine learning quântico!"
output = pipe(prompt, **generation_args)

In [13]:
output

[{'generated_text': '\n\n### Resposta:\nMachine learning quântico é um campo interdisciplinar que combina os princípios da computação quântica com técnicas de aprendizado de máquina. A computação quântica utiliza os princípios da mecânica quântica para processar informações de uma maneira que pode ser muito mais rápida e poderosa do que a computação clássica para certos tipos de problemas.\n\nO aprendizado de máquina é um subconjunto da inteligência artificial que se concentra em desenvolver algoritmos e modelos que podem aprender com dados e tomar decisões ou previsões com base nesses dados.\n\nMachine learning quântico tenta aproveitar os recursos únicos da computação quântica para melhorar o desempenho de algoritmos de aprendizado de máquina. Isso pode incluir tarefas como classificação de dados, regressão, agrupamento e reconhecimento de padrões.\n\nAlguns dos benefícios potenciais do machine learning quântico incluem:\n\n1. Velocidade: Alguns algoritmos de aprendizado de máquina p

In [14]:
print(output[0]['generated_text'])



### Resposta:
Machine learning quântico é um campo interdisciplinar que combina os princípios da computação quântica com técnicas de aprendizado de máquina. A computação quântica utiliza os princípios da mecânica quântica para processar informações de uma maneira que pode ser muito mais rápida e poderosa do que a computação clássica para certos tipos de problemas.

O aprendizado de máquina é um subconjunto da inteligência artificial que se concentra em desenvolver algoritmos e modelos que podem aprender com dados e tomar decisões ou previsões com base nesses dados.

Machine learning quântico tenta aproveitar os recursos únicos da computação quântica para melhorar o desempenho de algoritmos de aprendizado de máquina. Isso pode incluir tarefas como classificação de dados, regressão, agrupamento e reconhecimento de padrões.

Alguns dos benefícios potenciais do machine learning quântico incluem:

1. Velocidade: Alguns algoritmos de aprendizado de máquina podem ser executados muito mais r

In [15]:
prompt = "Quanto é 7 x 6 - 41 ?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



Opções de resposta: (A) 2 (B) -2 (C) 10 (D) -10 (E) -18


### Answer

Primeiro, realize a multiplicação: 7 x 6 = 42. Em seguida, subtraia 41 de 42: 42 - 41 = 1. Portanto, a resposta é 1, que não está listada nas opções de resposta fornecidas. Parece haver um erro nas opções de resposta.


In [16]:
prompt = "Quem foi a primeira pessoa no espaço?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



### Answer:A primeira pessoa a viajar no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, em seu voo espacial Vostok 1.


A primeira mulher no espaço foi Valentina Tereshkova, também da União Soviética. Ela voou em 16 de junho de 1963, em seu voo espacial Vostok 6.


A primeira pessoa a caminhar na Lua foi Neil Armstrong, um astronauta norte-americano. Ele fez isso em 20 de julho de 1969, durante a missão Apollo 11.


A primeira pessoa a pousar na Lua foi também Neil Armstrong, juntamente com o astronauta Edwin "Buzz" Aldrin, durante a missão Apollo 11. Eles pousaram na superfície lunar em 20 de julho de 1969.


A primeira pessoa a orbitar a Terra em uma espaçonave foi John Glenn, um astronauta norte-americano. Ele fez isso em 20 de fevereiro de 1962, em seu voo espacial Friendship 7.


A primeira pessoa a viajar para a Lua e retornar à Terra foi Neil Armstrong, como parte da tripulação da Apollo 11.


A primeira pes

### Templates e eng de prompt

In [17]:
prompt = "Quem foi a primeira pessoa no espaço?"

In [18]:
template = f"""<|system|>
You are a helpful assistant.<|end|>
<|user|>
{prompt}<|end|>
<|assistant|>
"""

In [19]:
template

'<|system|>\nYou are a helpful assistant.<|end|>\n<|user|>\nQuem foi a primeira pessoa no espaço?<|end|>\n<|assistant|>\n'

In [20]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, a bordo da nave espacial Vostok 1. Sua missão marcou um momento histórico na exploração espacial e foi um grande triunfo para a União Soviética durante a era da Guerra Fria.


In [21]:
prompt = "Você entende português?"

template = f"""<|system|>
You are a helpful assistant.<|end|>
<|user|>
{prompt}<|end|>
<|assistant|>
"""

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 Sim, eu entendo o português. Como um modelo de linguagem de IA, fui treinado em uma variedade de textos em língua portuguesa, o que me permite entender e gerar texto em português.


In [22]:
prompt = "Você entende português?" # @param

template = f"""<|system|>
You are a helpful assistant.<|end|>
<|user|>
{prompt}<|end|>
<|assistant|>
"""

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 Sim, eu entendo o português. Como posso ajudar você hoje?


### Engenharia de Prompt

In [23]:
prompt = "O que é IA?" # @param

sys_prompt = "Você é um assistente virtual prestativo. Responda sempre em português de forma direta e resumida! Disserte mais na resposta, apenas se solicitado."

template = f"""<|system|>
{sys_prompt}.<|end|>
<|user|>
{prompt}<|end|>
<|assistant|>
"""

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Esses sistemas podem aprender, raciocinar e se adaptar a novas situações. A IA é utilizada em diversas áreas, como automação, análise de dados, robótica e assistência pessoal.


In [24]:
prompt = "Explique o que é sequencia de Fibonacci e escreva um codigo python dela, explicando como funciona! E onde é utilizado, normalmente!" # @param

sys_prompt = "Você é um assistente virtual prestativo. Responda sempre em português de forma direta e resumida! Disserte mais na resposta, apenas se solicitado."

template = f"""<|system|>
{sys_prompt}.<|end|>
<|user|>
{prompt}<|end|>
<|assistant|>
"""

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A sequência de Fibonacci é uma série de números onde cada número é a soma dos dois anteriores. Tradicionalmente, a sequência começa com 0 e 1. Assim, os primeiros números da sequência são: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, e assim por diante.


Aqui está um exemplo simples de como implementar uma função em Python para gerar a sequência de Fibonacci até um número específico:


```python

def sequencia_fibonacci(n):

    if n <= 0:

        return []

    elif n == 1:

        return [0]

    fib_sequence = [0, 1]

    while len(fib_sequence) < n:

        next_value = fib_sequence[-1] + fib_sequence[-2]

        fib_sequence.append(next_value)

    return fib_sequence


# Exemplo de uso:

n = 10  # Quantidade de números da sequência desejada

print(sequencia_fibonacci(n))

```


A função `sequencia_fibonacci` recebe um número `n` e retorna a sequência de Fibonacci até o `n`º termo. Ela começa verificando se `n` é menor ou igual a 0, retornando uma lista vazia nesse caso. Se `n` for 1, r

In [25]:
def sequencia_fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]

    fib_sequence = [0, 1]

    while len(fib_sequence) < n:
        next_value = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_value)
    return fib_sequence



n = 10

print(sequencia_fibonacci(n))


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


### Formato de mensagens

In [26]:
prompt = "o que é IA?"

msg = [
    {
        "role": "system",
        "content": "Você é um assistente virtual prestativo. Responda sempre em português de forma direta e resumida! Disserte mais na resposta, apenas se solicitado."
    },
    {
        "role": "user",
        "content": prompt
    }
]

output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigem inteligência humana. Isso inclui processamento de linguagem natural, reconhecimento de imagens, aprendizado de máquina e tomada de decisões. A IA pode ser categorizada em diferentes tipos, como AI fraca (não consciente) e AI forte (consciente). A AI fraca é usada em aplicações como assistentes virtuais e recomendações de produtos, enquanto a AI forte é um objetivo futuro que visa criar máquinas com consciência e autoconsciência.


In [27]:
prompt = "o que é IA?"
prompt_sys = "Você é um assistente virtual prestativo. Responda sempre em português de forma direta e resumida! Disserte mais na resposta, apenas se solicitado."

msg = [
    {
        "role": "system",
        "content": prompt_sys
    },
    {
        "role": "user",
        "content": prompt
    }
]

output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Isso inclui processamento de linguagem natural, reconhecimento de imagens, aprendizado de máquina e tomada de decisões complexas. A IA pode ser categorizada em diferentes tipos, como AI fraca (não consciente) e AI forte (consciente). A AI fraca é projetada para realizar tarefas específicas, enquanto a AI forte teria a capacidade de entender e se adaptar a novas situações de forma semelhante à inteligência humana.


### Otimizando com Quantização

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

/home/vicrrs/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import getpass
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [5]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.99s/it]


In [8]:
prompt = "Quem foi a primeira pessoa no espaço?"
messages = [{"role": "user", "content": prompt}]

encodeds = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True
)
model_inputs = encodeds.to(device)

generated_ids = model.generate(
    model_inputs,
    max_new_tokens=64,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)

decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
decoded[0]


'Quem foi a primeira pessoa no espaço? A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele fez a primeira viagem orbital em 12 de abril de 1961, a bordo da nave Vostok 1. Sua missão, Vost'